In [1]:
import random
import numpy as np

random.seed(1234)
np.random.seed(1234)

# Dataset Statistics for Perception Package Projects
This example notebook shows how to use datasetinsights to load synthetic datasets generated from the [Perception package](https://github.com/Unity-Technologies/com.unity.perception) and visualize dataset statistics. It includes statistics and visualizations of the outputs built into the Perception package and should give a good idea of how to use datasetinsights to visualize custom annotations and metrics.

## Setup dataset
If the dataset was generated locally, point `data_root` below to the path of the dataset. The `GUID` folder suffix should be changed accordingly.   

In [2]:
data_root = "/home/igormaurell/Workspace/athome/unity_datasets/3e1b6176-12e1-41c8-8123-650e5378e142"

### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the metrics needed for dataset statistics.

Provide the `run-execution-id` which generated the dataset and a valid `access_token` in the following cell. The `access_token` can be generated using the Unity Simulation [CLI](https://github.com/Unity-Technologies/Unity-Simulation-Docs/blob/master/doc/cli.md#usim-inspect-auth).

In [3]:
from datasetinsights.io.downloader import UnitySimulationDownloader

#run execution id:
# run_execution_id = "xxx"
# #access_token:
# access_token = "xxx"
# #annotation definition id:
# annotation_definition_id = "6716c783-1c0e-44ae-b1b5-7f068454b66e"
# #unity project id
# project_id = "xxx"
# source_uri = f"usim://{project_id}/{run_execution_id}"

# downloader = UnitySimulationDownloader(access_token=access_token)


Before loading the dataset metadata for statistics we first download the relevant files from Unity Simulation.


In [4]:
# downloader.download(source_uri=source_uri, output=data_root)

## Load dataset metadata
Once the dataset metadata is downloaded, it can be loaded for statistics using `datasetinsights.data.simulation`. Annotation and metric definitions are loaded into pandas dataframes using `AnnotationDefinitions` and `MetricDefinitions` respectively.

In [5]:
from datasetinsights.datasets.unity_perception import AnnotationDefinitions, MetricDefinitions
ann_def = AnnotationDefinitions(data_root)
ann_def.table

,id,name,description,format,spec
0,f9f22e05-443f-4602-a422-ebe4ea9b55cb,bounding box,Bounding box for each labeled object visible t...,json,"[{'label_id': 0, 'label_name': 'ycb_003_cracke..."
1,1ccebeb4-5886-41ff-8fe0-f911fa8cbcdf,instance segmentation,pixel-wise instance segmentation label,PNG,"[{'label_id': 0, 'label_name': 'ycb_003_cracke..."


In [6]:
metric_def = MetricDefinitions(data_root)
metric_def.table

,id,name,description
0,db1b258e-d1d0-41b6-8751-16f601a2e230,scenario_iteration,Iteration information for dataset sequences
1,14adb394-46c0-47e8-a3f0-99e754483b76,random-seed,The random seed used to initialize the random ...


## Built-in Statistics
The following tables and charts are supplied by `datasetinsights.data.datasets.statistics.RenderedObjectInfo` on datasets that include the "rendered object info" metric.

In [7]:
from datasetinsights.stats.statistics import RenderedObjectInfo
import datasetinsights.datasets.unity_perception.metrics as metrics
from datasetinsights.datasets.unity_perception.exceptions import DefinitionIDError
from datasetinsights.stats import bar_plot, histogram_plot, rotation_plot

max_samples = 1000          # maximum number of samples points used in histogram plots

rendered_object_info_definition_id = "5ba92024-b3b7-41a7-9d3f-c03a6a8ddd01"
roinfo = None
try:
    roinfo = RenderedObjectInfo(data_root=data_root, def_id=rendered_object_info_definition_id)
except DefinitionIDError:
    print("No RenderedObjectInfo in this dataset")

No RenderedObjectInfo in this dataset


### Descriptive Statistics

In [8]:
if roinfo is not None:
    print(roinfo.num_captures())
    roinfo.raw_table.head(3)

### Total Object Count

In [9]:
if roinfo is not None:
    total_count = roinfo.total_counts()
    display(total_count)
    
    display(bar_plot(
        total_count, 
        x="label_id", 
        y="count", 
        x_title="Label Name",
        y_title="Count",
        title="Total Object Count in Dataset",
        hover_name="label_name"
    ))

### Per Capture Object Count

In [10]:
if roinfo is not None:
    per_capture_count = roinfo.per_capture_counts()
    display(per_capture_count.head(10))

In [11]:
if roinfo is not None:
    display(histogram_plot(
        per_capture_count, 
        x="count",  
        x_title="Object Counts Per Capture",
        y_title="Frequency",
        title="Distribution of Object Counts Per Capture",
        max_samples=max_samples
    ))

### Object Visible Pixels

In [12]:
if roinfo is not None:
    display(histogram_plot(
        roinfo.raw_table, 
        x="visible_pixels",  
        x_title="Visible Pixels Per Object",
        y_title="Frequency",
        title="Distribution of Visible Pixels Per Object",
        max_samples=max_samples
    ))

## Annotation Visualization
In the following sections we show how to load annotations from the Captures object and visualize them. Similar code can be used to consume annotations for model training or visualize and train on custom annotations.

### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the images, captures and annotations in the dataset. Make sure you have enough disk space to store all files. For example, a dataset with 100K captures requires roughly 300GiB storage.

In [13]:
# downloader.download(source_uri=source_uri, output=data_root, include_binary=True)

### Load captures

In [14]:
from datasetinsights.datasets.unity_perception.captures import Captures
cap = Captures(data_root)
cap.captures.head(3)

,id,sequence_id,step,timestamp,sensor,ego,filename,format,annotations
0,d59d25f9-3522-41a2-876e-3f13a670002f,e5fb3185-46fc-4077-92d0-6ad263222379,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': '56f669d3-1517-4cd5-b495-d6fb79056a49'...
1,6a6843b8-1360-4af8-a9d2-d0ed8197f50a,178011e3-47eb-4ec6-9778-dc766e03afa4,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': 'f116bb63-409f-44e8-8e84-3b7d4b98661d'...
2,365cb45e-a0c6-4af2-9c9e-46eea0011f1f,52bda99d-ffd7-4f69-b42e-6a36d8a16881,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': '7c069902-aed7-4caf-a151-a331f0706215'...


### Bounding Boxes
In this section we render 2d bounding boxes on top of the captured images.

In [15]:
import os

from ipywidgets import interact, interactive, fixed, interact_manual
from PIL import Image

from datasetinsights.stats.visualization.plots import plot_bboxes
from datasetinsights.datasets.synthetic import SynDetection2D,read_bounding_box_2d

bounding_box_definition_id = "f9f22e05-443f-4602-a422-ebe4ea9b55cb"
dataset = SynDetection2D(data_path=data_root, def_id=bounding_box_definition_id)
def draw_bounding_boxes(index):
    image, bboxes = dataset[index]
    return plot_bboxes(image, bboxes, dataset.label_mappings)

In [16]:
from ipywidgets import interact

# pick an index and visualize
interact(draw_bounding_boxes, index=list(range(len(dataset))))

interactive(children=(Dropdown(description='index', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,…

<function __main__.draw_bounding_boxes(index)>

### 3D Ground Truth Bounding Boxes
In this section we render 3d ground truth bounding boxes on top of the captured images.

In [17]:
import os
import numpy as np

from ipywidgets import interact
from PIL import Image
from datasetinsights.stats.visualization.plots import plot_bboxes3d
from datasetinsights.datasets.synthetic import read_bounding_box_3d

bounding_box_3d_defintion_id = "f9f22e05-443f-4602-a422-ebe4ea9b55cb"
def draw_bounding_boxes3d(index):
    filename = os.path.join(data_root, box_captures.loc[index, "filename"])
    annotations = box_captures.loc[index, "annotation.values"]
    sensor = box_captures.loc[index, "sensor"]

    if 'camera_intrinsic' in sensor:
        projection = np.array(sensor["camera_intrinsic"])
    else:
        projection = np.array([[1,0,0],[0,1,0],[0,0,1]])

    image = Image.open(filename)
    boxes = read_bounding_box_3d(annotations)
    img_with_boxes = plot_bboxes3d(image, boxes, projection)
    img_with_boxes.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_boxes)

try:
    pass
    #box_captures = cap.filter(def_id=bounding_box_3d_defintion_id)
    #interact(draw_bounding_boxes3d, index=(0, box_captures.shape[0]))
except DefinitionIDError:
    print("No bounding boxes found")

## Semantic Segmentation
In this section we render the semantic segmentation images on top of the captured images.

In [18]:
def draw_with_segmentation(index, opacity):
    filename = os.path.join(data_root, seg_captures.loc[index, "filename"])
    seg_filename = os.path.join(data_root, seg_captures.loc[index, "annotation.filename"])
    
    image = Image.open(filename)
    seg = Image.open(seg_filename)
    img_with_seg = Image.blend(image, seg, opacity)
    img_with_seg.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_seg)
    
try:
    semantic_segmentation_definition_id = "12f94d8d-5425-4deb-9b21-5e53ad957d66"
    seg_captures = cap.filter(def_id=semantic_segmentation_definition_id)
    interact(draw_with_segmentation, index=(0, seg_captures.shape[0]), opacity=(0.0, 1.0))
except DefinitionIDError:
    print("No semantic segmentation images found")

No semantic segmentation images found


## Instance Segmentation
In this section we render the instance segmentation images on top of the captured images. Image IDs are mapped to an RGBA color value, below the image we include a preview of the mapping between colors and IDs.

In [19]:
def instance_sorter(instance):
    return instance["instance_id"]

def draw_with_instance_segmentation(index, opacity):
    filename = os.path.join(data_root, inst_caps.loc[index, "filename"])
    seg_filename = os.path.join(data_root, inst_caps.loc[index, "annotation.filename"])

    image = Image.open(filename)
    seg = Image.open(seg_filename)
    img_with_seg = Image.blend(image, seg, opacity)
    img_with_seg.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_seg)

    anns = inst_caps.loc[index, "annotation.values"].copy()
    anns.sort(key=instance_sorter)

    count = min(5, len(anns))
    print("First {} ID entries:".format(count))

    for i in range(count):
        color = anns[i].get("color")
        print ("{} => Color({:>3}, {:>3}, {:>3})".format(anns[i].get("instance_id"), color.get("r"), color.get("g"), color.get("b")))

try:
    inst_seg_def_id = "1ccebeb4-5886-41ff-8fe0-f911fa8cbcdf"
    inst_caps = cap.filter(def_id=inst_seg_def_id)
    interact(draw_with_instance_segmentation, index=(0, inst_caps.shape[0]), opacity=(0.0, 1.0))
except DefinitionIDError:
    print("No instance segmentation images found")


interactive(children=(IntSlider(value=5000, description='index', max=10000), FloatSlider(value=0.5, descriptio…

In [20]:
bbox_captures = cap.filter(bounding_box_definition_id)
inst_captures = cap.filter(inst_seg_def_id)

In [21]:
bbox_captures.head()

,id,sequence_id,step,timestamp,sensor,ego,filename,format,annotations,annotation.id,annotation.annotation_definition,annotation.values,annotation.filename
0,d59d25f9-3522-41a2-876e-3f13a670002f,e5fb3185-46fc-4077-92d0-6ad263222379,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': '56f669d3-1517-4cd5-b495-d6fb79056a49'...,56f669d3-1517-4cd5-b495-d6fb79056a49,f9f22e05-443f-4602-a422-ebe4ea9b55cb,"[{'label_id': 12, 'label_name': 'ycb_004_sugar...",NaN
1,6a6843b8-1360-4af8-a9d2-d0ed8197f50a,178011e3-47eb-4ec6-9778-dc766e03afa4,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': 'f116bb63-409f-44e8-8e84-3b7d4b98661d'...,f116bb63-409f-44e8-8e84-3b7d4b98661d,f9f22e05-443f-4602-a422-ebe4ea9b55cb,"[{'label_id': 13, 'label_name': 'ycb_005_tomat...",NaN
2,365cb45e-a0c6-4af2-9c9e-46eea0011f1f,52bda99d-ffd7-4f69-b42e-6a36d8a16881,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': '7c069902-aed7-4caf-a151-a331f0706215'...,7c069902-aed7-4caf-a151-a331f0706215,f9f22e05-443f-4602-a422-ebe4ea9b55cb,"[{'label_id': 0, 'label_name': 'ycb_003_cracke...",NaN
3,48bc69cb-8212-4d8a-bf9f-3ce2efcdf2d2,1953fb6d-bda2-4e74-9add-d664d4bf839a,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': 'b8e270fc-fe17-40eb-9b9c-66ec8e0ae489'...,b8e270fc-fe17-40eb-9b9c-66ec8e0ae489,f9f22e05-443f-4602-a422-ebe4ea9b55cb,"[{'label_id': 9, 'label_name': 'ycb_007_tuna_f...",NaN
4,02b16a47-4738-4f55-84fb-0af508637e5f,b787623e-bd4f-4b47-99d1-02e3c9c7b268,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': '566fc617-7a40-4dad-bbb0-7f4970cbbdeb'...,566fc617-7a40-4dad-bbb0-7f4970cbbdeb,f9f22e05-443f-4602-a422-ebe4ea9b55cb,"[{'label_id': 10, 'label_name': 'ycb_002_maste...",NaN


In [22]:
inst_captures.head()

,id,sequence_id,step,timestamp,sensor,ego,filename,format,annotations,annotation.id,annotation.annotation_definition,annotation.values,annotation.filename
0,d59d25f9-3522-41a2-876e-3f13a670002f,e5fb3185-46fc-4077-92d0-6ad263222379,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': '56f669d3-1517-4cd5-b495-d6fb79056a49'...,152d93d4-98cb-4dc9-9612-99ddd9b6b2af,1ccebeb4-5886-41ff-8fe0-f911fa8cbcdf,"[{'instance_id': 38, 'color': {'r': 255, 'g': ...",InstanceSegmentationb74d83a2-0be7-4b13-89b4-d9...
1,6a6843b8-1360-4af8-a9d2-d0ed8197f50a,178011e3-47eb-4ec6-9778-dc766e03afa4,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': 'f116bb63-409f-44e8-8e84-3b7d4b98661d'...,b65206ea-c083-444e-955b-25ce50e5b271,1ccebeb4-5886-41ff-8fe0-f911fa8cbcdf,"[{'instance_id': 62, 'color': {'r': 51, 'g': 0...",InstanceSegmentationb74d83a2-0be7-4b13-89b4-d9...
2,365cb45e-a0c6-4af2-9c9e-46eea0011f1f,52bda99d-ffd7-4f69-b42e-6a36d8a16881,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': '7c069902-aed7-4caf-a151-a331f0706215'...,58963791-fd45-40f1-b8be-3240e689b1c8,1ccebeb4-5886-41ff-8fe0-f911fa8cbcdf,"[{'instance_id': 107, 'color': {'r': 240, 'g':...",InstanceSegmentationb74d83a2-0be7-4b13-89b4-d9...
3,48bc69cb-8212-4d8a-bf9f-3ce2efcdf2d2,1953fb6d-bda2-4e74-9add-d664d4bf839a,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': 'b8e270fc-fe17-40eb-9b9c-66ec8e0ae489'...,282050d3-d221-4422-b22a-515f8b4a008c,1ccebeb4-5886-41ff-8fe0-f911fa8cbcdf,"[{'instance_id': 36, 'color': {'r': 0, 'g': 54...",InstanceSegmentationb74d83a2-0be7-4b13-89b4-d9...
4,02b16a47-4738-4f55-84fb-0af508637e5f,b787623e-bd4f-4b47-99d1-02e3c9c7b268,0,0.0,{'sensor_id': '27864388-937d-4bce-b657-5d28d37...,{'ego_id': '3ae70021-18dc-4a95-b2a3-f7606b7eaa...,RGB268c3b64-677c-439b-a0d8-fc8c5f393156/rgb_15...,PNG,[{'id': '566fc617-7a40-4dad-bbb0-7f4970cbbdeb'...,9761bd40-1202-4fe1-8a4f-a5b7cc603f79,1ccebeb4-5886-41ff-8fe0-f911fa8cbcdf,"[{'instance_id': 105, 'color': {'r': 79, 'g': ...",InstanceSegmentationb74d83a2-0be7-4b13-89b4-d9...


In [23]:
dataset = []
for image_filename, bbox_ann, inst_ann, inst_mask in zip(bbox_captures['filename'], bbox_captures['annotation.values'], inst_captures['annotation.values'], inst_captures['annotation.filename']):
    dataset.append((image_filename, bbox_ann, inst_ann, inst_mask))

In [24]:
len(dataset)

10000

In [25]:
import random
random.shuffle(dataset)

In [26]:
train_size = int(0.9*len(dataset))
test_size = len(dataset) - train_size

In [27]:
train_set = dataset[:train_size]
test_set = dataset[train_size:]

In [28]:
import cv2
import numpy as np
import json
import tqdm

In [29]:
def write_coco_annotations(dataset, subset):
    coco_dict = {}
    coco_dict['info'] = {}
    coco_dict['images'] = []
    '''coco_dict['categories'] = [
        {
            "id": 0,
            "name": "BG",
            "supercategory": "background"
        }
    ]'''
    coco_dict['categories'] = []
    coco_dict['annotations'] = []
    labels = {}
    for i, (image_filename, bbox_ann, inst_ann, inst_mask) in enumerate(tqdm.tqdm(dataset)):
        img = Image.open(os.path.join(data_root, image_filename)).convert("RGB")
        mask = Image.open(os.path.join(data_root, inst_mask)).convert("RGB")
        img_arr = np.array(img)
        mask_arr = np.array(mask)
        for j in range(len(bbox_ann)):
            if bbox_ann[j]['label_id'] not in labels:
                labels[bbox_ann[j]['label_id']] = bbox_ann[j]['label_name']
            color = inst_ann[j]['color']
            mask_color = np.array([color['r'], color['g'], color['b']])
            object_mask = (mask_arr[:,:] == mask_color).all(axis=2)*255
            contours, hierarchy = cv2.findContours(object_mask.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if len(contours) > 0:
                bbox = [
                    float(bbox_ann[j]['x']),
                    float(bbox_ann[j]['y']),
                    float(bbox_ann[j]['width']),
                    float(bbox_ann[j]['height']),
                ]
                annotation = {}
                annotation['segmentation'] = [contour.reshape(-1).astype('float64').tolist() for contour in contours]
                annotation['area'] = int((np.sum(object_mask)/255)/3)
                annotation['iscrowd'] = 0
                annotation['bbox'] = bbox
                annotation['category_id'] = bbox_ann[j]['label_id']
                annotation['image_id'] = i
                annotation['id'] = i*len(dataset) + j
                coco_dict['annotations'].append(annotation)
        coco_dict['images'].append({
            'id': i,
            'width': img_arr.shape[1],
            'height': img_arr.shape[0],
            'file_name': image_filename
        })
    label_list = list(labels.items())
    label_list.sort(key=lambda x: x[0])
    for label_id, label_name in label_list:
        coco_dict['categories'].append({
            "id": label_id,
            "name": label_name,
            "supercategory": "household_object"
        })
    with open(os.path.join(data_root, "{}.json".format(subset)), 'w') as f:
        json.dump(coco_dict, f, indent=4, sort_keys=True)
    print(tuple([v for k, v in label_list]))

In [30]:
write_coco_annotations(train_set, "train")

100%|██████████| 9000/9000 [16:38<00:00,  9.02it/s]


('ycb_003_cracker_box', 'ycb_009_gelatin_box', 'ycb_008_pudding_box', 'ycb_010_potted_meat_can', 'ycb_006_mustard_bottle', 'ycb_018_plum', 'ycb_016_pear', 'ycb_011_banana', 'ycb_012_strawberry', 'ycb_007_tuna_fish_can', 'ycb_002_master_chef_can', 'ycb_014_lemon', 'ycb_004_sugar_box', 'ycb_005_tomato_soup_can', 'ycb_015_peach', 'ycb_013_apple', 'ycb_017_orange', 'person_standing')


In [31]:
write_coco_annotations(test_set, "test")

100%|██████████| 1000/1000 [02:03<00:00,  8.07it/s]


('ycb_003_cracker_box', 'ycb_009_gelatin_box', 'ycb_008_pudding_box', 'ycb_010_potted_meat_can', 'ycb_006_mustard_bottle', 'ycb_018_plum', 'ycb_016_pear', 'ycb_011_banana', 'ycb_012_strawberry', 'ycb_007_tuna_fish_can', 'ycb_002_master_chef_can', 'ycb_014_lemon', 'ycb_004_sugar_box', 'ycb_005_tomato_soup_can', 'ycb_015_peach', 'ycb_013_apple', 'ycb_017_orange', 'person_standing')


In [32]:
len(('ycb_003_cracker_box', 'ycb_009_gelatin_box', 'ycb_008_pudding_box', 'ycb_010_potted_meat_can', 'ycb_006_mustard_bottle', 'ycb_018_plum', 'ycb_016_pear', 'ycb_011_banana', 'ycb_012_strawberry', 'ycb_007_tuna_fish_can', 'ycb_002_master_chef_can', 'ycb_014_lemon', 'ycb_004_sugar_box', 'ycb_005_tomato_soup_can', 'ycb_015_peach', 'ycb_013_apple', 'ycb_017_orange', 'person_standing'))

18

In [34]:
a = (('ycb_028_skillet_lid', 'ycb_003_cracker_box', 'ycb_058_golf_ball', 'ycb_009_gelatin_box', 'ycb_073-b_lego_duplo', 'ycb_008_pudding_box', 'ycb_010_potted_meat_can', 'ycb_072-b_toy_airplane', 'ycb_062_dice', 'ycb_024_bowl', 'ycb_059_chain', 'ycb_006_mustard_bottle', 'ycb_018_plum', 'ycb_063-b_marbles', 'ycb_063-a_marbles', 'ycb_065-e_cups', 'ycb_016_pear', 'ycb_065-b_cups', 'ycb_073-c_lego_duplo', 'ycb_043_phillips_screwdriver', 'ycb_052_extra_large_clamp', 'ycb_071_nine_hole_peg_test', 'ycb_065-d_cups', 'ycb_061_foam_brick', 'ycb_025_mug', 'ycb_040_large_marker', 'ycb_065-f_cups', 'ycb_065-c_cups', 'ycb_054_softball', 'ycb_070-a_colored_wood_blocks', 'ycb_029_plate', 'ycb_030_fork', 'ycb_011_banana', 'ycb_072-c_toy_airplane', 'ycb_065-a_cups', 'ycb_035_power_drill', 'ycb_051_large_clamp', 'ycb_012_strawberry', 'ycb_007_tuna_fish_can', 'ycb_057_racquetball', 'ycb_044_flat_screwdriver', 'ycb_002_master_chef_can', 'ycb_048_hammer', 'ycb_072-e_toy_airplane', 'ycb_033_spatula', 'ycb_065-h_cups', 'ycb_073-e_lego_duplo', 'ycb_014_lemon', 'ycb_065-i_cups', 'ycb_050_medium_clamp', 'trofast', 'ycb_022_windex_bottle', 'ycb_053_mini_soccer_ball', 'ycb_004_sugar_box', 'ycb_072-a_toy_airplane', 'ycb_070-b_colored_wood_blocks', 'ycb_005_tomato_soup_can', 'ycb_065-g_cups', 'ycb_015_peach', 'ycb_027_skillet', 'ycb_077_rubiks_cube', 'ycb_013_apple', 'ycb_032_knife', 'ycb_056_tennis_ball', 'ycb_073-d_lego_duplo', 'ycb_038_padlock', 'ycb_019_pitcher_base', 'ycb_073-a_lego_duplo', 'ycb_065-j_cups', 'ycb_072-d_toy_airplane', 'ycb_037_scissors', 'ycb_036_wood_block', 'ycb_026_sponge', 'ycb_031_spoon', 'ycb_055_baseball', 'ycb_017_orange', 'ycb_021_bleach_cleanser', 'ycb_073-f_lego_duplo', 'ycb_042_adjustable_wrench', 'ycb_073-g_lego_duplo'))
b = (('person_standing', 'ycb_002_master_chef_can', 'ycb_003_cracker_box', 'ycb_004_sugar_box', 'ycb_005_tomato_soup_can', 'ycb_006_mustard_bottle', 'ycb_007_tuna_fish_can', 'ycb_008_pudding_box', 'ycb_009_gelatin_box', 'ycb_010_potted_meat_can', 'ycb_011_banana', 'ycb_012_strawberry', 'ycb_013_apple', 'ycb_014_lemon', 'ycb_015_peach', 'ycb_016_pear', 'ycb_017_orange', 'ycb_018_plum', 'ycb_019_pitcher_base', 'ycb_021_bleach_cleanser', 'ycb_022_windex_bottle', 'ycb_024_bowl', 'ycb_025_mug', 'ycb_026_sponge', 'ycb_029_plate', 'ycb_030_fork', 'ycb_031_spoon', 'ycb_033_spatula', 'ycb_040_large_marker', 'ycb_051_large_clamp', 'ycb_053_mini_soccer_ball', 'ycb_054_softball', 'ycb_055_baseball', 'ycb_056_tennis_ball', 'ycb_057_racquetball', 'ycb_058_golf_ball', 'ycb_059_chain', 'ycb_061_foam_brick', 'ycb_062_dice', 'ycb_063-a_marbles', 'ycb_063-b_marbles', 'ycb_065-a_cups', 'ycb_065-b_cups', 'ycb_065-c_cups', 'ycb_065-d_cups', 'ycb_065-e_cups', 'ycb_065-f_cups', 'ycb_065-g_cups', 'ycb_065-h_cups', 'ycb_065-i_cups', 'ycb_065-j_cups', 'ycb_070-a_colored_wood_blocks', 'ycb_070-b_colored_wood_blocks', 'ycb_071_nine_hole_peg_test', 'ycb_072-a_toy_airplane', 'ycb_072-b_toy_airplane', 'ycb_072-c_toy_airplane', 'ycb_072-d_toy_airplane', 'ycb_072-e_toy_airplane', 'ycb_073-a_lego_duplo', 'ycb_073-b_lego_duplo', 'ycb_073-c_lego_duplo', 'ycb_073-d_lego_duplo', 'ycb_073-e_lego_duplo', 'ycb_073-f_lego_duplo', 'ycb_073-g_lego_duplo', 'ycb_077_rubiks_cube'))
a == b

True